In [8]:
import numpy as np
from numpy.linalg import solve

In [15]:
#閉鎖型ネットワークのノード到着率αを求める
def getCloseTraffic(p):
    e = np.eye(len(p)-1) #次元を1つ小さくする
    pe = p[1:len(p), 1:len(p)].T - e #行と列を指定して次元を小さくする
    lmd = p[0, 1:len(p)] #0行1列からの値を右辺に用いる
    try:
        slv = solve(pe, lmd * (-1)) #2021/09/28 ここで逆行列がないとエラーが出る
    except np.linalg.LinAlgError as err: #2021/09/29 Singular Matrixが出た時は、対角成分に小さい値を足すことで対応 https://www.kuroshum.com/entry/2018/12/28/python%E3%81%A7singular_matrix%E3%81%8C%E8%B5%B7%E3%81%8D%E3%82%8B%E7%90%86%E7%94%B1
        print('Singular Matrix')
        pe += e * 0.00001 
        slv = solve(pe, lmd * (-1)) 
    #lmd *= -1
    #slv = np.linalg.pinv(pe) * lmd #疑似逆行列で求める
    alpha = np.insert(slv, 0, 1.0) #α1=1を追加
    return alpha    

In [26]:
def getT(m, mu, k):
  for i in range(len(mu)):
    if m[i] == 1: #窓口数1
      T[i] = 1 / mu[i] * (1 + L[i])
    if m[i] == 0 : #無限サーバ(IS)
      T[i] = 1 / mu[i]
    if m[i] > 1 : #複数窓口
      val = 1 / (mu[i] * m[i])
      tmp = 1 + L[i]
      for j in range(m[i]-2 + 1):
        tmp += (m[i] - j - 1) * getPi(i, j, k - 1)
      val *= tmp
      T[i] = val  

In [27]:
def getPi(i, a, b):
  if a == 0 and b == 0:
    return pi[i][a][b]
  if a == 0 and b > 0: #(8.36)
    tmp = alpha[i] * Lambda[b-1] / mu[i]
    for j in range(1, m[i] -1 + 1):
      tmp += (m[i] -j) * getPi(i, j, b) #再帰呼び出し
    print('i = {0}, a = {1}, b = {2}, pi = {3}'.format(i, a, b, 1 - 1 / m[i] *tmp))
    return 1 - 1 / m[i] * tmp
  else: #(8.37)
    alp = m[i]
    if a <= m[i]:#P387上部
      alp = a
    print('i = {0}, a = {1}, b = {2}, pi = {3}'.format(i, a, b, Lambda[b-1] / (mu[i] * alp) * getPi(i, a-1, b-1)))
    return Lambda[b-1] / (mu[i] * alp) * getPi(i, a-1, b-1) #再帰呼び出し

In [28]:
def getLambda(alpha, k):
  tmp = 0
  for i in range(len(alpha)):
    tmp += alpha[i] * T[i]
  #Lambda = 1 / tmp
  return k / tmp

In [29]:
def getL(alpha, k):
  for i in range(len(alpha)):
    L[i] = Lambda[k-1] * T[i] * alpha[i]

In [30]:
#初期設定
K = 3
m = [2, 1, 1, 0] #Node4はIS(Infinite server)
mu = [2.0, 5/3, 5/4, 1.0]
p = np.array([[0, 0.5, 0.5, 0],
              [0, 0, 0, 1],
              [0, 0, 0, 1],
              [1, 0, 0, 0]
              ])
#Step1 (k = 0)
L = np.zeros(len(p))
T = np.zeros(len(p))
Lambda = np.zeros(K)
pi = np.zeros((len(p), K, K))
pi[:][0][0] = 1
print('pi = {}'.format(pi))
#到着率を求める
alpha = getCloseTraffic(p)
print('alpha = {}'.format(alpha))

#k = 1
k = 1
getT(m, mu, k)
print('T({0}) = {1}'.format(k, T))
Lambda[k-1] = getLambda(alpha, k)
print('Lambda({0}) = {1}'.format(k, Lambda[k-1]))
getL(alpha, k)
print('L({0}) = {1}'.format(k, L))

#k = 2
k = 2
getT(m, mu, k)
print('T({0}) = {1}'.format(k, T))
Lambda[k-1] = getLambda(alpha, k)
print('Lambda({0}) = {1}'.format(k, Lambda[k-1]))
getL(alpha, k)
print('L({0}) = {1}'.format(k, L))

#ここまでOK k >= 3のとき値が違っているので確認する (2022/06/13)
#多分計算済みのλが更新されてしまっているので、それを保存して使うか、piを保存する
#k = 3
k = 3
getT(m, mu, k)
print('T({0}) = {1}'.format(k, T))
Lambda[k-1] = getLambda(alpha, k)
print('Lambda({0}) = {1}'.format(k, Lambda[k-1]))
getL(alpha, k)
print('L({0}) = {1}'.format(k, L))

pi = [[[1. 1. 1.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]]
alpha = [1.  0.5 0.5 1. ]
T(1) = [0.5 0.6 0.8 1. ]
Lambda(1) = 0.45454545454545453
L(1) = [0.22727273 0.13636364 0.18181818 0.45454545]
i = 0, a = 1, b = 1, pi = 0.22727272727272727
i = 0, a = 0, b = 1, pi = 0.7727272727272727
T(2) = [0.5        0.68181818 0.94545455 1.        ]
Lambda(2) = 0.8644400785854618
L(2) = [0.43222004 0.29469548 0.4086444  0.86444008]
i = 0, a = 1, b = 1, pi = 0.22727272727272727
i = 0, a = 0, b = 1, pi = 0.7727272727272727
i = 0, a = 1, b = 2, pi = 0.3339882121807466
i = 0, a = 1, b = 1, pi = 0.22727272727272727
i = 0, a = 0, b = 1, pi = 0.7727272727272727
i = 0, a = 0, b = 2, pi = 0.6168958742632613
T(3) = [0.51227898 0.77681729 1.12691552 1.        ]
Lambda(3) = 1.2174606338449274
L(3) = [0.62367949 0.47287223 0.68598764 1.21746063]
